In [0]:
from IPython.display import clear_output
!wget "https://raw.githubusercontent.com/newtonis/PASA/master/Guia_2/The-Pink-Panther-Theme-Song.wav"
clear_output()

from scipy.io.wavfile import read
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import wave, struct, math, random

In [0]:
#para leer stereo
file_name = "The-Pink-Panther-Theme-Song.wav"  
file = read(file_name)
print(file)
datos = np.array(file[1],dtype=float)
ch0 = datos[:, 0]
ch1 = datos[:, 1]

(48000, array([], dtype=int16))


In [12]:
file_name = "The-Pink-Panther-Theme-Song.wav"  

def read_whole(filename):
    wav_r = wave.open(filename, 'r')
    ret = []
    while wav_r.tell() < wav_r.getnframes():
        decoded = struct.unpack("<h", wav_r.readframes(1))
        ret.append(decoded)
    return ret

archivo = read_whole(file_name)
print(archivo)

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size/2:]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

